# Spacy exploration

TODO:

- ~Explore RoBERTa for NER.~
- ~Try text examples that are more complex and derived from the fashion domain.~

Experiment status:

(Oct. 18th) T-shirt is not being recognized as category, 'white and pink stripes' contains color and pattern information that is not being recognized. Stains and pilling also not picked up, but this is not surprising.

In [4]:
import spacy
from spacy import displacy


# spacy.cli.download("en_core_web_sm")
# spacy.cli.download("en_core_web_trf")

## Choose model: Vanilla small English model vs transformer (RoBERTa)

In [19]:
# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_trf')
# doc = nlp(u'Tesla is looking at buying U.S. startup for $6 million')
fashion_text = """A H&M t-shirt for ladies, size xl, couple of stains, pilling 1, condition 3, white and pink stripes, 100% cotton, made in China, 2019, $5.00
"""
doc = nlp(fashion_text)


for token in doc:
    print(token.text, token.pos_, token.dep_, token.ent_type_)
    # print(dir(token))

A DET det 
H&M PROPN compound ORG
t NOUN compound 
- PUNCT punct 
shirt NOUN ROOT 
for ADP prep 
ladies NOUN pobj 
, PUNCT punct 
size NOUN compound 
xl NOUN appos 
, PUNCT punct 
couple NOUN appos 
of ADP prep 
stains NOUN pobj 
, PUNCT punct 
pilling NOUN appos 
1 NUM nummod CARDINAL
, PUNCT punct 
condition NOUN appos 
3 NUM nummod CARDINAL
, PUNCT punct 
white ADJ amod 
and CCONJ cc 
pink ADJ conj 
stripes NOUN appos 
, PUNCT punct 
100 NUM nummod PERCENT
% NOUN compound PERCENT
cotton NOUN appos 
, PUNCT punct 
made VERB acl 
in ADP prep 
China PROPN pobj GPE
, PUNCT punct 
2019 NUM npadvmod DATE
, PUNCT punct 
$ SYM nmod 
5.00 NUM appos MONEY

 SPACE dep 


In [3]:
# displacy.serve(doc, style="ent")

# For Jupyter Notebook
displacy.render(doc, style="ent")

## Using a `Matcher`

In [24]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

# Define patterns
brand_pattern = [{"LOWER": {"IN": ["h&m", "zara", "nike"]}}]
size_pattern = [{"LOWER": {"IN": ["small", "medium", "large", "xl", "40", "42"]}}]
# condition_pattern = [{"IS_DIGIT": True, {"IN": ["1", "2", "3", "4", "5"]}}]
# condition_pattern = [{"LIKE_NUM": True, "ORTH": {"IN": ["1", "2", "3", "4", "5"]}}]
condition_pattern = [{"LOWER": "condition"}, {"LIKE_NUM": True, "ORTH": {"IN": ["1", "2", "3", "4", "5"]}}]

# Add patterns to matcher
matcher.add("BRAND", [brand_pattern])
matcher.add("SIZE", [size_pattern])
matcher.add("CONDITION", [condition_pattern])

In [26]:
text = "This is an H&M t-shirt, size medium, condition 4."
# doc = nlp(text)
doc = nlp(fashion_text)
matches = matcher(doc)

# Extract matched entities
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(f"{string_id}: {span[-1].text}")

BRAND: H&M
SIZE: xl
CONDITION: 3
